# KHUDA 9기 ML세션
2026.02.19 (목)

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [ ]:
# dataset repository path
url = "https://raw.githubusercontent.com/gaeng02/KHUDA-9th-ML/main/week5/dataset.csv"

In [ ]:
#df = pd.read_csv(url)
df = pd.read_csv("./dataset.csv")
df.head()

In [ ]:
### answer init
answer = [0] * 5

## Task 0. 초기 설정
다음 설정에 맞게 데이터셋을 준비하세요. <br>
별도의 정답은 없지만, 완료되지 않은 경우 Task 수행 불가

- feature: `income_k, spend_score, age, visits_m`
- scaling: `StandardScaler()`

In [ ]:
## Input Box

feature_cols = ["income_k", "spend_score", "age", "visits_m"]
X = df[feature_cols].values
scaler = StandardScaler()
X = scaler.fit_transform(X)

## Task 1
Kmeans 모델에서 Silhouette을 이용하여 best k를 찾으세요. <br>
k의 범위는 2이상 8이하로, 가장 Silhouette Score가 큰 k를 `answer[1]`에 저장하세요. <br>
(만약, Silhouette score가 동일하다면, 작은 k를 선택) <br><br>

이때, `random_state = 219, n_init = 10`로 설정하세요.

In [ ]:
## Input Box

best_k = None
best_score = 0.0

for k in range (2, 9) :
    km = KMeans(n_clusters = k, random_state = 219, n_init = 10)
    labels = km.fit_predict(X)
    s = silhouette_score(X, labels)
    if s > best_score:
        best_score = s
        best_k = k


answer[1] = best_k
print(answer[1])

## Task 2
Task 1에서 구한 k 값으로 Kmeans를 학습하세요. <br>
클러스터 중 `spend_score` 평균이 가장 큰 클러스터의 sample 수를 `answer[2]`에 저장하세요. <br><br>

이때, `random_state = 219, n_init = 10`로 설정하세요.
<br><br>
[Hint] cluster별로 label을 추가해준 df_copy를 만들고 `groupby` 사용하기

In [ ]:
## Input Box

count = 0
kmeans = KMeans(n_clusters = answer[1], random_state = 219, n_init = 10)
labels = kmeans.fit_predict(X)
tmp = df.copy()
tmp["cluster"] = labels
vip_cluster = tmp.groupby("cluster")["spend_score"].mean().idxmax()
count = (tmp['cluster'] == vip_cluster).sum()


answer[2] = count
print(answer[2])

## Task 3
Task 2에서 학습한 Kmeans에 대해 `cluster_centers_`를 이용해, 서로 다른 두 중심 사이의 유클리드 거리 중 최대값을 구하세요. <br>
이때 최대 거리를 `answer[3]`에 저장하세요.

In [ ]:
## Input Box

max_distance = 0.0
centers = kmeans.cluster_centers_

for i in range(len(centers)):
    for j in range(i+1, len(centers)):
        d = float(np.linalg.norm(centers[i]-centers[j]))
        if (d > max_distance) : max_distance = d

answer[3] = max_distance
print(answer[3])

## Task 4
DBSCAN으로 군집을 만들고, 노이즈(클러스터에 속하지 못함)의 수를 구하세요. <br>
이때, 노이즈의 수를 `answer[4]`에 저장하세요.
<br><br>
이때, `eps = 0.5, min_samples = 5`로 설정하세요.

In [ ]:
## Input Box

count = 0

db = DBSCAN(eps = 0.5, min_samples = 5)
db_labels = db.fit_predict(X)
count = (db_labels == -1).sum()

answer[4] = count
print(answer[4])

# 정답 확인!

In [ ]:
print("=== 작성하신 정답 ===")
for i in range (1, 5) : print("Task " + str(i) + " :: " + str(answer[i]))

In [ ]:
### 정답 채점 코드
import hashlib

ANSWER_URL = "https://raw.githubusercontent.com/gaeng02/KHUDA-9th-ML/main/week5/answer.csv"

def md5 (s) : return hashlib.md5(s.encode("utf-8")).hexdigest()

ans_df = pd.read_csv(ANSWER_URL)
gt = {int(r["task"]): str(r["answer"]).strip().lower()
      for _, r in ans_df.iterrows()}

wrong = []

for i in range(1, 5) :
    user_answer = "" if answer[i] is None else md5(str(answer[i]).strip().lower())
    if (user_answer != gt.get(i, "")) : wrong.append(i)

print("탈출!" if not wrong else f"틀린번호 : {wrong}")